In [1]:
import chess
import logging 
import os

os.unlink("currentMatch.log")
logging.basicConfig(filename="currentMatch.log", encoding="utf-8", level=logging.INFO, force=True)

def log_new_position(board: chess.Board) -> None:
    commentTurn = "after black turn" if board.turn == chess.WHITE else "after white turn"
    logging.info(f"new boarding fen {commentTurn}: {board.fen()}")
    display(board)

In [2]:
import chess.engine as ce
import IPython.display as Id

board = chess.Board()
#board.set_board_fen("r1b5/pp1n1k2/1q2p2p/3p4/3PnP1P/P4N2/1PQ2PP1/2R2RK1")
engine = ce.SimpleEngine.popen_uci("stockfish")
limit = ce.Limit(time=0.2)

gamingMoves = []
nextMove = input("Please enter your next move:")
while nextMove != "quit":
    if nextMove=="back":
        if len(gamingMoves)>1:
            nextMove = gamingMoves[-2]
            gamingMoves.pop()
            board.pop()
        else:
            board = chess.Board()
            gamingMoves = []
        log_new_position(board)
    else:
        try:    
            move = chess.Move.from_uci(nextMove)
            if move in board.legal_moves:
                board.push_uci(nextMove)
                gamingMoves.append(nextMove)
                log_new_position(board)
                if board.is_checkmate():
                    logging.info("Checkmate! Game Ended")
                    break
                suggestions = engine.analyse(board, limit, multipv=3)
                for suggestion in suggestions:
                    print("score:",suggestion["score"],"suggested move:",suggestion["pv"][0])
            else:
                print("This is not a legal move or quit, please enter a valid move")
            print(gamingMoves)
        except chess.InvalidMoveError as err:
            print(f"Invalid Move {nextMove}. Please try one of the followings: {[chess.Move.uci(legal_move) for legal_move in board.legal_moves]}")
    
    nextMove = input("Please enter your next move:")
    Id.clear_output()
